In [1]:
import pandas as pd

In [2]:
file = "data/tblTokensNew.xlsx"

In [3]:
df = pd.read_excel(file)

In [11]:
# create necessary concept-schemes
ecce_pos,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-pos")

In [12]:
lemmas = df[['lemma', 'lemmaPoS']]

In [13]:
lemmas = lemmas.drop_duplicates().values.tolist()

In [14]:
pos = df[['PoS', 'lemmaPoS']]

In [15]:
pos = set(pos['PoS'].drop_duplicates().values.tolist() + pos['lemmaPoS'].drop_duplicates().values.tolist())

In [16]:
pos = set([x.upper() for x in list(pos) if isinstance(x,str)])

In [17]:
for x in pos:
    temp_variable,_ = SkosConcept.objects.get_or_create(pref_label=x)
    temp_variable.save()
    temp_variable.scheme = [ecce_pos]

In [18]:
for x in lemmas:
    temp_lemma,_ = Lemma.objects.get_or_create(name=x[0], pos=SkosConcept.objects.get_or_create(pref_label=x[1])[0])
    temp_lemma.save()

In [50]:
sample = df.ix[280994:340000]

In [51]:
# ceate objects
for i, row in sample.iterrows():
    temp_object,_ = Token.objects.get_or_create(
        legacy_id=row['ID'],
        spelling=row['spelling'],
        left_context=row['leftcontext'],
        plain_word=row['plainword'],
        comments=row['comments'],
        right_context=row['rightcontext'],
        file=row['file'],
        medial_suffix=row['medialsuffix'],
        final_suffix=row['finalsuffix'],
        weight = row['weight'],
        weight_norm = row['weightPerMilPerSemi']
    )
    temp_lemma,_ = Lemma.objects.get_or_create(
            name=row['lemma'], pos=SkosConcept.objects.get_or_create(pref_label=row['lemmaPoS'])[0]
        )
    temp_object.lemma = temp_lemma
    temp_object.save()
    try:
        temp_pos = SkosConcept.objects.get(pref_label=row['PoS'])
        temp_object.pos=temp_pos
        temp_object.save()
    except:
        pass
    try:
        temp_fk = TokenLabel.objects.get(label=row['label'])
        temp_object.label=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Text.objects.get(text=row['textsource'])
        temp_object.text_source=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Cluster.objects.get(consonant=row['cluster'])
        temp_object.cluster=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = SchwaPresent.objects.get(spelling=row['spelling2'])
        temp_object.spelling2=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = OnSet.objects.get(rightonset=row['rightonset'])
        temp_object.rightonset=temp_fk
        temp_object.save()
    except:
        pass

In [52]:
# number of tokens
Token.objects.all().count()

331439

In [41]:
#for x in Token.objects.all():
   # x.delete()